In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Importing Required Libraries
import numpy as np
import colorcet
import bokeh.io
import bokeh.plotting
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import bokeh.layouts
import bokeh.models

bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
def simulate_AND_gate(ranges, ktrs):
    miR21_0 = []
    miR92_0 = []
    k_tr = []
    y = []
    for i in ranges:
        print(i)
        i = i*5
        for j in ranges:
            print(j)
            j = j*5
            for k in ktrs:
#                 k = k*7.79
                y0 = [i, j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
                # 0 miR21_1, 0 miR92_1, 0 CLS, 0 ols_1, 0 ols_2, 0 ols_c, 0 aeB_1, aeB_2, aeB_c, aeB_t
                ts = 43200
                t, params = get_params(k, ts)
                res = odeint(sim_AND, y0, t, args=(params,))
                res = np.append(res, np.full((ts, 1), 3.86e-5), axis=1)
                y.append(res)
                miR21_0.extend([i]*len(y[0]))
                miR92_0.extend([j]*len(y[0]))
                k_tr.extend([k]*len(y[0]))
    return(miR21_0, miR92_0, k_tr, y)

In [4]:
def simulate_OR_gate(ranges, ktrs):
    miR21_0 = []
#     miR92_0 = []
    k_tr = []
    y = []
    for i in ranges:
            print(i)
            i = i*5
            for k in ktrs:
#                 k = k*7.79
                y0 = [i, 0, 0, 0, 0]  # 0 miR21_0, miR21_1, 0 CLS, 0 ols, aeB_t
                ts = 43200
                t, params = get_params(k, ts)
                res = odeint(sim_OR, y0, t, args=(params,))
                res = np.append(res, np.full((ts, 1), 3.86e-5), axis=1)
                y.append(res)
                miR21_0.extend([i]*len(y[0]))
                k_tr.extend([k]*len(y[0]))
    return(miR21_0, k_tr, y)

In [5]:
def get_params(k_tr, ts):
  # Setting Initial Conditions and Time Points
  # miR21_0, miR92_0, miR21_1, miR92_1, CLS, ols_1, ols_2, ols_c, aeB
  # miR21_0 = 0.001
  # miR92_0 = 0.001

    t = np.linspace(0, ts, num=ts)

    # Defining Parameters and ODE System
    # miR21_1 fluctuation
    k_di21 = 1.e-2
    k_de21 = 3.e-4
    k_21_CLS = 1.e5
    #k_21_CLS = 1.e7

    # miR92_1 fluctuation
    k_di92 = 1.e-2
    k_de92 = 3.e-4
    k_92_CLS = 1.e5

    # CLS fluctuation
    k_tr = 0.000779
    #k_tr = 0.0000779
    #k_tr = 0.00000779
    #k_tr = 0.0000000779
    k_de_CLS = 3.e-4
    #k_de_CLS = 3.e-2

    # OLS-1 fluctuation
    k_ols_1_92 = 1.e5
    k_de_OLS_1 = 0.0003

    # parameters of OLS-2
    k_ols_2_21 = 1.e5
    k_de_OLS_2 = 0.0003

    # parameters of OLS-c
    k_de_OLS_c = 0.0003

    # parameters of AeBlue fluctuation
    k_tl = 0.03441 
    k_de_aeb = 0.00027

    params = [k_di21, k_de21, k_21_CLS, k_di92, k_de92, k_92_CLS, k_tr, k_de_CLS,
              k_ols_1_92, k_de_OLS_1, k_ols_2_21, k_de_OLS_2, k_de_OLS_c, k_tl, k_de_aeb]

    return(t, params)

In [6]:
def sim_AND(variables, t, params):
    # variables
    miR21_0 = variables[0]
    miR92_0 = variables[1]
    miR21_1 = variables[2]
    miR92_1 = variables[3]
    cls = variables[4]
    ols_1 = variables[5]
    ols_2 = variables[6]
    ols_c = variables[7]
    aeb_1 = variables[8]
    aeb_2 = variables[9]
    aeb_c = variables[10]
    
    # parameters of miR21 fluctuation
    k_di21 = params[0]
    k_de21 = params[1]
    k_21_CLS = params[2]

    # parameters of miR92 fluctuation
    k_di92 = params[3]
    k_de92 = params[4]
    k_92_CLS = params[5]

    # parameters of CLS fluctuation
    k_tr = params[6]
    k_de_CLS = params[7]

    # parameters of OLS-1
    k_ols_1_92 = params[8]
    k_de_OLS_1 = params[9]

    # parameters of OLS-2
    k_ols_2_21 = params[10]
    k_de_OLS_2 = params[11]

    # parameters of OLS-c
    k_de_OLS_c = params[12]

    # parameters of AeBlue fluctuation
    k_tl = params[13]
    k_de_aeb = params[14]

    # Equations
    dmiR21_0dt = -k_di21 * miR21_0
    dmiR92_0dt = -k_di92 * miR92_0
   
    dmiR21_1dt = k_di21 * miR21_0 - k_de21 * miR21_1 - k_21_CLS * cls * miR21_1 - k_ols_2_21 * ols_2 * miR21_1
    dmiR92_1dt = k_di92 * miR92_0 - k_de92 * miR92_1 - k_92_CLS * cls * miR92_1 - k_ols_1_92 * ols_1 * miR92_1
    
    dCLSdt = k_tr - k_de_CLS * cls - k_21_CLS * cls * miR21_1 - k_92_CLS * cls * miR92_1
    dOLS_1dt = k_21_CLS * cls * miR21_1 - k_ols_1_92 * ols_1 * miR92_1 - k_de_OLS_1 * ols_1
    dOLS_2dt = k_92_CLS * cls * miR92_1 - k_ols_2_21 * ols_2 * miR21_1 - k_de_OLS_2 * ols_2
    dOLS_cdt = k_ols_1_92 * ols_1 * miR21_1 + k_ols_2_21 * ols_2 * miR21_1 - k_de_OLS_c * ols_c
    daeb_1dt = k_tl * ols_1 - k_de_aeb * aeb_1
    daeb_2dt = k_tl * ols_2 - k_de_aeb * aeb_2
    daeb_cdt = k_tl * ols_c - k_de_aeb * aeb_c
    daeb_tdt = daeb_1dt + daeb_2dt + daeb_cdt

    return [dmiR21_0dt, dmiR92_0dt, dmiR21_1dt, dmiR92_1dt, dCLSdt, dOLS_1dt, dOLS_2dt, dOLS_cdt, daeb_1dt, daeb_2dt, daeb_cdt, daeb_tdt]

In [7]:
def sim_OR(variables, t, params):
    # variables
    miR21_0 = variables[0]
    # miR92_0 = variables[1]
    miR21_1 = variables[1]
    # miR92_1 = variables[3]
    cls = variables[2]
    ols_1 = variables[3]
    # ols_2 = variables[5]
    # ols_c = variables[7]
    aeb_1 = variables[4]
    # aeb_2 = variables[9]
    # aeb_c = variables[10]

    # parameters of miR21 fluctuation
    k_di21 = params[0]
    k_de21 = params[1]
    k_21_CLS = params[2]

    # parameters of miR92 fluctuation
    k_di92 = params[3]
    k_de92 = params[4]
    k_92_CLS = params[5]

    # parameters of CLS fluctuation
    k_tr = params[6]
    k_de_CLS = params[7]

    # parameters of OLS-1
    # k_ols_1_92 = params[8]
    k_de_OLS_1 = params[9]

    # parameters of OLS-2
    k_ols_2_21 = params[10]
    k_de_OLS_2 = params[11]

    # parameters of OLS-c
    k_de_OLS_c = params[12]

    # parameters of AeBlue fluctuation
    k_tl = params[13]
    k_de_aeb = params[14]

    # Equations
    dmiR21_0dt = -k_di21 * miR21_0

    dmiR21_1dt = k_di21 * miR21_0 - k_de21 * miR21_1 - k_21_CLS * cls * miR21_1
    
    dCLSdt = k_tr - k_de_CLS * cls - k_21_CLS * cls * miR21_1
    dOLS_dt = k_21_CLS * cls * miR21_1 - k_de_OLS_1 * ols_1
    daeb_dt = k_tl * ols_1 - k_de_aeb * aeb_1


    return [dmiR21_0dt, dmiR21_1dt, dCLSdt, dOLS_dt, daeb_dt]

In [8]:
def get_df_AND(y):
    # Create a DataFrame
    df = pd.DataFrame()
    
    # Create a list of column names
    column_names = ['miR21_0', 'miR92_0', 'miR21_1', 'miR92_1', 'CLS', 'ols_1', 'ols_2', 'ols_c', 'aeB_1', 'aeB_2', 'aeB_c', 'aeB_t', 'aeB-visible']

    # Create a list of row indexes
    row_indexes = [y[:, 0], y[:, 1], y[:, 2], y[:, 3], y[:, 4], y[:, 5], y[:, 6], y[:, 7], y[:, 8], y[:, 9], y[:, 10], y[:, 11], y[:, 12]]


    for i in range(len(column_names)):
        df[column_names[i]] = row_indexes[i]
    return(df)

In [9]:
def get_df_OR(y):
    # Create a DataFrame
    df = pd.DataFrame()
    
    # Create a list of column names
    column_names = ['miR21_0', 'miR21_1', 'CLS', 'ols', 'aeB_t','aeB_visible']

    # Create a list of row indexes
    row_indexes = [y[:,0], y[:,1], y[:,2], y[:,3], y[:,4], y[:,5]]

    for i in range(len(column_names)):
        df[column_names[i]] = row_indexes[i]
    return(df)

In [10]:
import pandas as pd
start = -1  # Starting exponent, corresponds to 10^-3
stop = -4   # Ending exponent, corresponds to 10^-6
step = -1 

# Create a list of powers of 10 using list comprehension
ranges = [10.0 ** n for n in np.arange(start, stop + step, step)]
ktrs = [0.000779, 0.0000779, 0.00000779]
miR21_0, miR92_0, k_tr, y = simulate_AND_gate(ranges, ktrs)

# Create a DataFrame
df = pd.DataFrame()

for i in y:
    df = pd.concat([df,get_df_AND(i)])
df['miR21_0_init'] = miR21_0
df['miR92_0_init'] = miR92_0
df['k_tr'] = k_tr
df.to_csv('AND.csv', index = True)
df.head()

0.1
0.1
0.01
0.001
0.0001
0.01
0.1
0.01
0.001
0.0001
0.001
0.1
0.01
0.001
0.0001
0.0001
0.1
0.01
0.001
0.0001


,miR21_0,miR92_0,miR21_1,miR92_1,CLS,ols_1,ols_2,ols_c,aeB_1,aeB_2,aeB_c,aeB_t,aeB-visible,miR21_0_init,miR92_0_init,k_tr
0,0.500000,0.500000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000039,0.5,0.5,0.000779
1,0.495025,0.495025,0.004197,0.004197,9.298842e-07,9.322189e-07,9.322189e-07,0.000776,1.033545e-07,1.033545e-07,0.000013,0.000013,0.000039,0.5,0.5,0.000779
2,0.490099,0.490099,0.008341,0.008341,4.663612e-07,4.666237e-07,4.666237e-07,0.001556,1.255971e-07,1.255971e-07,0.000053,0.000053,0.000039,0.5,0.5,0.000779
3,0.485222,0.485222,0.012436,0.012436,3.121237e-07,3.122064e-07,3.122064e-07,0.002335,1.388003e-07,1.388003e-07,0.000120,0.000120,0.000039,0.5,0.5,0.000779
4,0.480394,0.480394,0.016480,0.016480,2.359265e-07,2.359789e-07,2.359789e-07,0.003113,1.480661e-07,1.480661e-07,0.000214,0.000214,0.000039,0.5,0.5,0.000779


In [11]:
print(df['miR21_0_init'].value_counts())
print(df['miR92_0_init'].value_counts())
print(df['k_tr'].value_counts())

miR21_0_init
0.5000    518400
0.0500    518400
0.0050    518400
0.0005    518400
Name: count, dtype: int64
miR92_0_init
0.5000    518400
0.0500    518400
0.0050    518400
0.0005    518400
Name: count, dtype: int64
k_tr
0.000779    691200
0.000078    691200
0.000008    691200
Name: count, dtype: int64


In [12]:
import pandas as pd
start = -1  # Starting exponent, corresponds to 10^-3
stop = -4  # Ending exponent, corresponds to 10^-6
step = -1 

# Create a list of powers of 10 using list comprehension
ranges = [10.0 ** n for n in np.arange(start, stop + step, step)]
ktrs = [0.000779, 0.0000779, 0.00000779]
miR21_0, k_tr, y = simulate_OR_gate(ranges, ktrs)

# Create a DataFrame
df = pd.DataFrame()

for i in y:
    df = pd.concat([df,get_df_OR(i)])
df['miR21_0_init'] = miR21_0
# df['miR92_0_init'] = miR92_0
df['k_tr'] = k_tr
df.to_csv('OR-1miR.csv', index = True)
df.head()

0.1
0.01
0.001
0.0001


,miR21_0,miR21_1,CLS,ols,aeB_t,aeB_visible,miR21_0_init,k_tr
0,0.500000,0.000000,0.000000e+00,0.000000,0.000000,0.000039,0.5,0.000779
1,0.495025,0.004197,1.858209e-06,0.000777,0.000013,0.000039,0.5,0.000779
2,0.490099,0.008341,9.343682e-07,0.001557,0.000053,0.000039,0.5,0.000779
3,0.485222,0.012436,6.264594e-07,0.002335,0.000120,0.000039,0.5,0.000779
4,0.480394,0.016480,4.717084e-07,0.003114,0.000214,0.000039,0.5,0.000779


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518400 entries, 0 to 43199
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   miR21_0       518400 non-null  float64
 1   miR21_1       518400 non-null  float64
 2   CLS           518400 non-null  float64
 3   ols           518400 non-null  float64
 4   aeB_t         518400 non-null  float64
 5   aeB_visible   518400 non-null  float64
 6   miR21_0_init  518400 non-null  float64
 7   k_tr          518400 non-null  float64
dtypes: float64(8)
memory usage: 35.6 MB


In [14]:
print(df['miR21_0_init'].value_counts())
print(df['k_tr'].value_counts())

miR21_0_init
0.5000    129600
0.0500    129600
0.0050    129600
0.0005    129600
Name: count, dtype: int64
k_tr
0.000779    172800
0.000078    172800
0.000008    172800
Name: count, dtype: int64
